# Projeto 1 - Ciência dos Dados

Nome: Maria Eduarda Torres

Nome: Gabriela Choichit Giosa

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Gabriela\Documents\INSPER\2Semestre.2\CDados\Projetotweeter


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
dados = pd.read_excel('Iphone_11.xlsx')
dados.head()

,Treinamento,categorias
0,rt @maionese53: papai acaba de ajeitar meu cel...,4
1,@kt_lilstar @tiktokproibido no fim qm vai ter ...,3
2,estou aceitando um iphone 11 pro max ou um s20...,3
3,ai n to fazendo nada acho q vou comprar um iph...,3
4,@_gabialmeida07 ganhando iphone 11 eu lambia a...,4


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

#### Separando os dados

In [4]:
GM = dados['categorias'] == 5
G = dados['categorias'] == 4
N = dados['categorias'] == 3
NG = dados['categorias'] == 2
D = dados['categorias'] == 1

In [5]:
Gosta_muito = dados.loc[GM,:]
Gosta = dados.loc[G,:]
Neutro = dados.loc[N,:]
Nao_gosta = dados.loc[NG,:]
Detesta = dados.loc[D,:]

In [6]:
universo = dados['categorias']
universo.value_counts()

3    156
5     94
4     24
2     16
1     10
Name: categorias, dtype: int64

##### Subcategorias:


In [7]:
#Tweets positivos
Ok = (dados['categorias'] == 5) | (dados['categorias'] == 4)
positivo = dados.loc[Ok,:]

In [8]:
#Tweets negativos
N_ok = (dados['categorias'] == 1) | (dados['categorias'] == 2)
negativo = dados.loc[N_ok,:]

In [9]:
#Tweets neutros
n = (dados['categorias'] == 3)
neutro = dados.loc[n,:]

##### Total de ocorrências

In [10]:
total = len(dados.index)
total_neg = len(negativo.index)
total_pos = len(positivo.index)
total_neutro = len(neutro.index)
total_pos, total, total_neg, total_neutro

(118, 300, 26, 156)

##### Probabilidade de ser positivo

In [11]:
prob_pos = total_pos / total
prob_pos

0.3933333333333333

##### Probabilidade de ser negativo

In [12]:
prob_neg = total_neg / total
prob_neg

0.08666666666666667

##### Probabilidade de ser neutro

In [13]:
prob_neutro = total_neutro / total
prob_neutro

0.52

##### Transformando em texto 

In [14]:
texto=""

In [15]:
for e in dados.Treinamento:
    texto+= " " + e 


##### Filtrando texto positivo

In [16]:
texto_positivo =""

In [17]:
for e in positivo.Treinamento:
    texto_positivo+= " " + e 


##### Filtrando texto negativo 

In [18]:
texto_negativo =""

In [19]:
for e in negativo.Treinamento:
    texto_negativo+= " " + e 
    

##### Filtrando texto neutro

In [20]:
texto_neutro =""

In [21]:
for e in neutro.Treinamento:
    texto_neutro+= "" + e 
    

##### Limpando textos


In [22]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed
    

##### Split and cleanup

In [23]:
import re

texto = cleanup(texto.lower())
texto_split = pd.Series(texto.split())

texto_positivo = cleanup(texto_positivo.lower())
texto_positivo_split = pd.Series(texto_positivo.split())

texto_negativo = cleanup(texto_negativo.lower())
texto_negativo_split = pd.Series(texto_negativo.split())

texto_neutro= cleanup(texto_neutro.lower())
texto_neutro_split = pd.Series(texto_neutro.split())

for e in dados.Treinamento:
    e = cleanup(e.lower())
    x = re.sub("@[A-Za-z0-9]+","",e)
x

'rt   eu quero pega o iphone 11 cansei desse já'

##### Frequencia absoluta

In [24]:
tabela_texto = texto_split.value_counts()

tabela_texto_positivo = texto_positivo_split.value_counts()

tabela_texto_negativo = texto_negativo_split .value_counts()

tabela_texto_neutro = texto_neutro_split.value_counts()

soma_palavras=tabela_texto.sum()
soma_positivo=tabela_texto_positivo.sum()
soma_negativo=tabela_texto_negativo.sum()
soma_neutro=tabela_texto_neutro.sum()

print(soma_palavras,soma_positivo,soma_negativo,soma_neutro)


probabilidade_pos=soma_positivo/ soma_palavras
probabilidade_neg=soma_negativo/ soma_palavras
probabilidade_neu=soma_neutro/ soma_palavras

print(probabilidade_pos,probabilidade_neg,probabilidade_neu)

5430 1703 551 3038
0.3136279926335175 0.10147329650092081 0.5594843462246777


##### Frequencias relativas

In [25]:
tabela_texto_relativa = texto_split.value_counts(True)

In [26]:
tabela_positivo_relativa = texto_positivo_split.value_counts(True)
tabela_positivo_relativa

iphone             0.073987
11                 0.070464
um                 0.042866
pro                0.022314
o                  0.022314
                     ...   
deus               0.000587
kylie              0.000587
apareceu           0.000587
@alemaiaoficial    0.000587
alcool             0.000587
Length: 575, dtype: float64

In [27]:
tabela_negativo_relativa = texto_negativo_split.value_counts(True)

In [28]:
tabela_neutro_relativa = texto_neutro_split.value_counts(True)

##### Probalidade

In [29]:
a=tabela_positivo_relativa.get('caro', 0)
b=tabela_negativo_relativa.get('caro', 0)
c=tabela_neutro_relativa.get('caro', 0)
a,b,c

(0.0005871990604815032, 0.0054446460980036296, 0)

##### Interseção entre os conjuntos

In [30]:
set_positivo = set(tabela_positivo_relativa.index)
set_negativo = set(tabela_negativo_relativa.index)
set_neutro = set(tabela_neutro_relativa.index)

inter = set_positivo.intersection(set_negativo, set_neutro)

##### Frase a se classificar

In [31]:
base_teste = pd.ExcelFile('Iphone_11.xlsx')
df1 = pd.read_excel(base_teste,'Teste')
df1.iloc[99]


Teste         rt @_matheusmaki: queria um iphone 11 e um app...
categorias                                                    5
Name: 99, dtype: object

##### Limpando tweets e classificando

In [32]:


i=0
for e in df1.Teste: 
    e = cleanup(e.lower())
    x = re.sub("@[A-Za-z0-9]+","",e)
    tabela_tweets = pd.Series(x.split())
    
    positive=1
    for palavra in tabela_tweets:
        positive*=tabela_positivo_relativa.get(palavra, 1)

    negative=1    
    for palavra in tabela_tweets:
        negative*=tabela_negativo_relativa.get(palavra, 1)
    
    cinza=1
    for palavra in tabela_tweets:
        cinza*=tabela_neutro_relativa.get(palavra, 1)
  
    
    positive=probabilidade_pos*positive
    negative=probabilidade_neg*negative
    cinza=probabilidade_neu*cinza
    
    if positive>negative and positive>cinza:
        classificador=5
    elif negative>positive and negative>cinza:
        classificador=1
    elif cinza>positive and cinza>negative:
        classificador=3
        
    #df1.insert(1, "classificador", classificador, True)
    df1.loc[i, "classificador"]=classificador
    i+=1
    
print(positive, negative,cinza)

print (df1)
    

1.5968404095906092e-40 6.609141688234601e-22 5.382731429352668e-44
                                                 Teste  categorias  \
0    rt @esteziinh_: @biaaa_crf marco mesmo pq não ...           5   
1    gabrieljnio6 @ericat_lol po man o meu é iphone...           3   
2        iphone 11 é difícil pra crl p mexer kkka🤦🏾‍♂️           1   
3    rt @xajuliaxouza: só preciso de um iphone 11 e...           5   
4    eu na #quarentena marcando pessoas em sorteio ...           3   
..                                                 ...         ...   
100  me da um iphone 11 na mao pra ve se eu nao vir...           5   
101  rt @arthurbclsz: alguém me dá um iphone xr? po...           5   
102  rt @euleozin_: troco por iphone 11 pro, chamar...           3   
103            o auto falante do iphone 11 pro é otimo           5   
104  @almeidac_11 @embaixadachina @zlj517 @spokespe...           3   

     classificador  
0              1.0  
1              3.0  
2              1.0  
3       

In [33]:
con = 0
total = 0
while con < 105:
    if df1.categorias[con] == df1.classificador[con]:
        total += 1
        
    con+=1 
total

13

In [34]:
porcentagem_acerto = total/len(df1)
porcentagem_acerto

0.12380952380952381

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

Neste trabalho buscamos fazer uma análise de sentimento referente as opiniões do público geral sobre o iphone 11. Para obter essas informações coletamos tweets que mencionassem o produto, para obtermos um resultado mais especifico, resolvemos nos basear em uma pergunta: Quais recursos poderíamos melhorar para o lançamento do próximo iphone?

   Desta forma, seguindo a pergunta, foi possível separar essa base de dados em cinco categorias: Gosto muito, Gosto, Neutro, Não gosto e Detesto. Porém avaliamos e visando melhorar a qualidade do nosso classificador, agrupamos o "Gosto muito" com o "Gosto", em uma classe chamada "Positivos", e "Não gosto" e "Detesto" em outra chamada "Negativos".

   Assim, os tweets Positivos, englobam aqueles que tem um feedback agregando os pontos bons do produto; Negativos, tweets que criticassem a qualidade do produto, desencorajando então pessoas a comprá-lo, e por fim, Neutros, que seriam aqueles que não respondessem a nossa pergunta, esse processo foi feito tanto na base de treinamento como na de teste.
   
   Começamos a divisão baseando-se na base de treinamento, onde já tinhamos as divisões e poderiamos montar os cálculos a partir delas, por conta disso, não poderiamos usar o classificador para gerar amostras na base de treinamento, uma vez que isso afetaria nossos resultados pois é de lá que retiramos os valores para treinar o programa.
   
   Com os tweets já classificados fizemos ajustes que melhorariam a perfomance de nosso classificador, começamos tranformando as células do dataframe em um texto corrido, para depois remover a pontuação e separar as palavras para podermos calcular a probabilidade de aparição de cada uma delas para futuramente às aplicar na fórmula naive bayes. Em seguida deixamos o texto todo em letra minuscula e julgamos interessante remover também os "@" e o nome de usuários.
   
   Com esses dados, o programa foi capaz de classificar sozinho nossa base de teste, levando em conta a probabilidade de cada palavra do tweet pertencer a determinada categoria para assim definir em qual delas esse tweet se encaixa. Nosso classificador teve uma acurácia de 12,38%, ou seja, a classe que nós colocamos os tweets foram iguais a categoria que o programa colocou em 12,38% dos casos.
   
   O Naive Bayes pode ser usado em diversas ocasiões, não só na classificação de tweets, mas como também para avaliar a probabilidade de eventos. Um exemplo de aplicação pode ser prever se um ex presidiário cometerá um crime novamente dependendo da cadeia em que foi preso. Outra possibilidade seria calcular a probabilidade de vitória de um jogador no jogo da velha, baseando-se na posição em que ele colocou a primeira peça.  

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**